<a href="https://www.kaggle.com/code/gpreda/endava-payments-hackathon-quick-starter?scriptVersionId=126252190" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

 <h1>Endava Payments Hackathon Quick Starter</h1>
 <h2>Connect to a Google Cloud account, create a BigQuery client, run queries, perform preliminary analysis</h2>
 
 This Kernel demonstrates the following:  
 - How to connect to a Google Cloud account;  
 - Initialize a BigQuery client;  
 - Get the tables list;  
 - Run queries to import BigQuery tables;  
 - Convert the tables to dataframes;  
 - Perform preliminary data evaluation using pandas profiling.
 

### Big Query Connector

In [1]:
from google.cloud import bigquery

### Connect to the project/dataset

In [2]:
bigquery_client = bigquery.Client(project='endava-hackathon')
dataset_ref = bigquery_client.dataset("payments_tx")

## Set configuration

In [3]:
config = {
    'debug': True,
    'train_sample': 0.05,
    'project': 'endava-hackathon',
    'database': 'endava-hackathon',
    'dataset': 'payments_tx'
}

config

{'debug': True,
 'train_sample': 0.05,
 'project': 'endava-hackathon',
 'database': 'endava-hackathon',
 'dataset': 'payments_tx'}

### Get the tables list

In [4]:
tables = list(bigquery_client.list_tables(dataset_ref))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

customers
merchants
terminals
transactions_complete
transactions_train


### Run a simple query


Get the shape of tables: 
- customers  
- merchants  
- terminals  
- transaction_train  

In [5]:
query = f"""SELECT COUNT(*) FROM `{config['database']}.{config['dataset']}.customers`"""
query_job = bigquery_client.query(query)
print(f"Customers: {query_job.to_dataframe().values[0][0]} rows.")

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Customers: 50000 rows.


In [6]:
query = f"""SELECT COUNT(*) FROM `{config['database']}.{config['dataset']}.merchants`"""
query_job = bigquery_client.query(query)
print(f"Merchants: {query_job.to_dataframe().values[0][0]} rows.")

Merchants: 30452 rows.


In [7]:
query = f"""SELECT COUNT(*) FROM `{config['database']}.{config['dataset']}.terminals`"""
query_job = bigquery_client.query(query)
print(f"Terminals: {query_job.to_dataframe().values[0][0]} rows.")

Terminals: 5000 rows.


In [8]:
query = f"""SELECT COUNT(*) FROM `{config['database']}.{config['dataset']}.transactions_train`"""
query_job = bigquery_client.query(query)
print(f"Transaction_train: {query_job.to_dataframe().values[0][0]} rows.")

transaction_train_full_shape = int(query_job.to_dataframe().values[0][0])

Transaction_train: 3393443 rows.


## Run queries to read tables


We read the tables and convert the tables to pandas dataframes.

In [9]:
import time
start_time = time.time()

query = f"""SELECT * FROM `{config['database']}.{config['dataset']}.customers`"""
query_job = bigquery_client.query(query)
customers_df = query_job.to_dataframe()
print(f"Query executed in: {round(time.time() - start_time, 2)} sec. for {customers_df.shape[0]} rows, {customers_df.shape[1]} cols.")

customers_df.head()

Query executed in: 9.83 sec. for 50000 rows, 3 cols.


,CUSTOMER_ID,x_customer_id,y_customer_id
0,9146778701554837,80.845748,58.465662
1,5298159883488498,56.205709,83.090319
2,1001491422509859,21.440831,41.026478
3,6293111408475835,31.060315,6.606118
4,5558521274211537,75.896886,81.742451


In [10]:
start_time = time.time()
query = f"""SELECT * FROM `{config['database']}.{config['dataset']}.merchants`"""
query_job = bigquery_client.query(query)
merchants_df = query_job.to_dataframe()
print(f"Query executed in: {round(time.time() - start_time, 2)} sec. for {merchants_df.shape[0]} rows, {merchants_df.shape[1]} cols.")
merchants_df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Query executed in: 20.46 sec. for 30452 rows, 24 cols.


,MERCHANT_ID,BUSINESS_TYPE,MCC_CODE,LEGAL_NAME,FOUNDATION_DATE,TAX_EXCEMPT_INDICATOR,VAT_NUMBER,TRADING_NAME,NICK_NAME,OUTLET_TYPE,...,AVERAGE_TICKET_SALE_AMOUNT,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE,DELIVERY_SAME_DAYS_PERCENTAGE,DELIVERY_WEEK_ONE_PERCENTAGE,DELIVERY_WEEK_TWO_PERCENTAGE,DELIVERY_OVER_TWO_WEEKS_PERCENTAGE
0,f3bb3b06-7f40-4fab-9936-4293a68d18cf,Limited Liability Company (LLC),5921,a0755c,2015-02-06,False,dbdb8a81-07f6-4257-bfa1-eaa9bb80135a,a0755c,a0755c,Face to face,...,104,14,57,29,0,0,29,25,26,20
1,487bdf4e-2104-4797-a8bf-d1ff90e8d7be,Corporations,4789,ab7ce2,2021-12-02,False,bc595505-1fdc-4c71-9221-630096f6d9f9,ab7ce2,ab7ce2,Ecommerce,...,201,27,59,14,0,0,24,25,22,29
2,5bb2f64b-525b-459a-8102-4aa31aca45f5,Sole Proprietorships,1750,deaecb,2016-09-02,False,f11b2cd7-43f0-4977-8912-d605c0156678,deaecb,deaecb,Ecommerce,...,92,28,43,29,0,0,23,20,21,36
3,c4790465-b17a-4ab5-a39a-81c0857151a7,S Corporations,8062,c2a2cd,2010-10-21,False,e1833cde-8354-4b8b-8a59-aaca5ce701b3,c2a2cd,c2a2cd,Face to face and ecommerce,...,157,29,62,9,0,0,26,28,26,20
4,d993e7d0-8daa-4618-9a01-45cea3c4a263,Limited Liability Company (LLC),5969,6b50c6,2013-04-27,False,4b9b6673-5ad0-4876-bc21-d2357815e0a8,6b50c6,6b50c6,Face to face,...,210,30,55,15,0,0,20,24,25,31


In [11]:
start_time = time.time()
query = f"""SELECT * FROM `{config['database']}.{config['dataset']}.terminals`"""
query_job = bigquery_client.query(query)
terminals_df = query_job.to_dataframe()
print(f"Query executed in: {round(time.time() - start_time, 2)} sec. for {terminals_df.shape[0]} rows, {terminals_df.shape[1]} cols.")
terminals_df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Query executed in: 1.53 sec. for 5000 rows, 3 cols.


,TERMINAL_ID,x_terminal_id,y_terminal__id
0,09754224,41.536730,68.889911
1,37467861,61.864075,20.882662
2,20678215,70.530677,36.567633
3,12724021,92.961874,66.721471
4,02989793,38.756717,68.669003


In [12]:
if config["debug"]:
    query = f"""SELECT * FROM `{config['database']}.{config['dataset']}.transactions_train` LIMIT {int(config['train_sample'] * transaction_train_full_shape)}"""
else:
    query = f"""SELECT * FROM `{config['database']}.{config['dataset']}.transactions_train`"""
start_time = time.time()    
query_job = bigquery_client.query(query)
transactions_train_df = query_job.to_dataframe()
print(f"Query executed in: {round(time.time() - start_time, 2)} sec. for {transactions_train_df.shape[0]} rows, {transactions_train_df.shape[1]} cols.")
transactions_train_df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Query executed in: 103.33 sec. for 169672 rows, 21 cols.


,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,CARD_EXPIRY_DATE,CARD_DATA,...,TRANSACTION_TYPE,TRANSACTION_STATUS,FAILURE_CODE,FAILURE_REASON,TRANSACTION_CURRENCY,CARD_COUNTRY_CODE,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD
0,b2bdb0d29b07925078ee975a823985924bab3c53,2021-04-27 14:51:44+00:00,8959186642301557,58419023,2.060000000,0,2.060000000,0E-9,11/21,6011********131,...,Purchase,Settled,None,None,AED,US,e7f13b95-4efb-495d-b278-dc0bfb4501bd,False,ACQ3,Offline enciphered PIN
1,311607316558f3fa3e85e5ade7f957f4b7fe9f32,2021-06-11 18:09:17+00:00,5810787734627040,36528632,1.840000000,0,1.840000000,0E-9,04/23,3742********727,...,Purchase,Settled,None,None,AED,US,138316ab-d5f1-41d2-b444-3c4819a03c2f,False,ACQ2,Offline plaintext PIN and signature
2,1eac6cc663ce5cc76d12507b9b187c45a69cef2b,2021-03-13 22:41:48+00:00,9744656952071047,42211721,0.840000000,0,0.840000000,0E-9,02/23,4716********928,...,Purchase,Settled,None,None,AED,US,09ec7a7d-d420-4013-96aa-1d9af66a2bf0,False,ACQ3,Online PIN
3,095b2806b98cb0d963ca70c83064798b2288e6f4,2021-10-21 14:03:57+00:00,5052466705246430,17142994,2.120000000,0,2.120000000,0E-9,04/22,5337********179,...,Purchase,Settled,None,None,AED,US,dbb60596-966d-4bd4-ba71-eec99232bb82,False,ACQ2,Online PIN
4,5371770143f2336eaa3acf4cb11a2a68784da4b7,2021-11-17 15:08:24+00:00,8395937513329700,15873857,0.760000000,0,0.760000000,0E-9,10/22,5337********185,...,Purchase,Settled,None,None,AED,US,f79a1501-3583-4fb3-91b6-c1d2ee1013a5,False,ACQ3,No CVM performed


### Query to get columns list

Get the columns names for `merchants` table.

In [13]:
query = f"""SELECT column_name FROM `{config['database']}.{config['dataset']}.INFORMATION_SCHEMA.COLUMNS` \
    WHERE table_name = 'merchants'"""

query_job = bigquery_client.query(query)
query_df = query_job.to_dataframe()
print(f"Columns in `merchants` table: {query_df.column_name.values}")

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Columns in `merchants` table: ['MERCHANT_ID' 'BUSINESS_TYPE' 'MCC_CODE' 'LEGAL_NAME' 'FOUNDATION_DATE'
 'TAX_EXCEMPT_INDICATOR' 'VAT_NUMBER' 'TRADING_NAME' 'NICK_NAME'
 'OUTLET_TYPE' 'ACTIVE_FROM' 'TRADING_FROM' 'ANNUAL_TURNOVER_CARD'
 'ANNUAL_TURNOVER' 'AVERAGE_TICKET_SALE_AMOUNT'
 'PAYMENT_PERCENTAGE_FACE_TO_FACE' 'PAYMENT_PERCENTAGE_ECOM'
 'PAYMENT_PERCENTAGE_MOTO' 'DEPOSIT_REQUIRED_PERCENTAGE'
 'DEPOSIT_PERCENTAGE' 'DELIVERY_SAME_DAYS_PERCENTAGE'
 'DELIVERY_WEEK_ONE_PERCENTAGE' 'DELIVERY_WEEK_TWO_PERCENTAGE'
 'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE']


Get the columns names for terminals table.

In [14]:
query = f"""SELECT column_name FROM `{config['database']}.{config['dataset']}.INFORMATION_SCHEMA.COLUMNS` \
    WHERE table_name = 'terminals'"""

query_job = bigquery_client.query(query)
query_df = query_job.to_dataframe()
print(f"Columns in `terminals` table: {query_df.column_name.values}")

Columns in `terminals` table: ['TERMINAL_ID' 'x_terminal_id' 'y_terminal__id']


## Data profiling using pandas_profiling

We use the option to run a minimal analysis.

In [15]:
from pandas_profiling import ProfileReport

In [16]:
profile = ProfileReport(customers_df, minimal=True)
profile.to_file(output_file="customers_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
profile = ProfileReport(merchants_df, minimal=True)
profile.to_file(output_file="merchants_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
profile = ProfileReport(terminals_df, minimal=True)
profile.to_file(output_file="terminals_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
profile = ProfileReport(transactions_train_df, minimal=True)
profile.to_file(output_file="transactions_train_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]